## Дискурсивный анализ текстов



## Дискурсивная сегментация


In [1]:
! pip install conllu


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
! wget https://www.dropbox.com/s/3rgfad28lh0m7w2/rstreebank_segmentation.zip && unzip rstreebank_segmentation.zip

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/peter/.wget-hsts'. HSTS will be disabled.
--2022-12-26 21:01:39--  https://www.dropbox.com/s/3rgfad28lh0m7w2/rstreebank_segmentation.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18, 2620:100:6026:18::a27d:4612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/3rgfad28lh0m7w2/rstreebank_segmentation.zip [following]
--2022-12-26 21:01:40--  https://www.dropbox.com/s/raw/3rgfad28lh0m7w2/rstreebank_segmentation.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc92e2e0f689719891f8fd652dc1.dl.dropboxusercontent.com/cd/0/inline/BzUYrNn447WAEjAdMY4X1fbiPLYH05bUIukYyg9Xdb3PeyRyD_1DZcBj5tbpVoczVCM6o6qhOnhmCBZd9vLWJFDm3rMfpK7_MRNdWjjJNqHYDCPSrTglWoOHALYheMJK65MV4iDEjAA7H-uejB-nlR0_Sb

In [1]:
! head -40 rus.rst.rrt_test.conll

# newdoc id = data_ru/test.blogs_93
1	_html_	_html_	X	_	Foreign=Yes	22	nsubj	_	BeginSeg=Yes
2	:	:	PUNCT	_	_	1	punct	_	_
3	/	/	PUNCT	_	_	1	punct	_	_
4	/	/	PUNCT	_	_	5	punct	_	_
5	ru-travel	ru-travel	X	_	Foreign=Yes	1	flat:foreign	_	_
6	.	.	PUNCT	_	_	1	punct	_	_
7	livejournal	livejournal	PROPN	_	Foreign=Yes	1	appos	_	_
8	.	.	PUNCT	_	_	1	punct	_	_
9	com	com	X	_	Foreign=Yes	1	appos	_	_
10	/	/	PUNCT	_	_	11	punct	_	_
11	33766823	33766823	NUM	_	_	9	appos	_	_
12	.	.	PUNCT	_	_	1	punct	_	_
13	html	html	PROPN	_	Foreign=Yes	22	nsubj	_	_

# newdoc id = data_ru/test.blogs_93
1	Маврикий	маврикий	PROPN	_	Animacу=Anim|Case=Nom|Gender=Masc|Number=Sing	13	appos	_	BeginSeg=Yes

# newdoc id = data_ru/test.blogs_93
1	После	после	ADP	_	_	16	case	_	BeginSeg=Yes
2	поездки	поездка	NOUN	_	Animacу=Inan|Case=Gen|Gender=Fem|Number=Sing	22	obl	_	_
3	в	в	ADP	_	_	18	case	_	_
4	2017	2017	ADJ	_	_	16	nmod	_	_
5	на	на	ADP	_	_	20	case	_	_
6	Бали	бали	PROPN	_	Animacу=Inan|Case=Loc|Gender=Masc|Number=Sing	16	nmod	_	_
7	я	я	P

In [2]:
import conllu

test_sentences = conllu.parse(open("rus.rst.rrt_test.conll").read())
train_sentences = conllu.parse(open("rus.rst.rrt_train.conll").read())

In [3]:
test_sentences[0]

TokenList<_html_, :, /, /, ru-travel, ., livejournal, ., com, /, 33766823, ., html, metadata={newdoc id: "data_ru/test.blogs_93"}>

In [4]:
print(len(test_sentences), len(train_sentences))

1568 11587


In [5]:
test_sentences[2][1]

{'id': 2,
 'form': 'поездки',
 'lemma': 'поездка',
 'upos': 'NOUN',
 'xpos': None,
 'feats': {'Animacу': 'Inan',
  'Case': 'Gen',
  'Gender': 'Fem',
  'Number': 'Sing'},
 'head': 22,
 'deprel': 'obl',
 'deps': None,
 'misc': None}

In [6]:
test_sent_list = list(test_sentences)
train_sent_list = list(train_sentences)

def get_labels(s_list):
    labels = []
    for sent in s_list:
        sent_labels = []
        for token in sent:
            if (token['misc'] != None):
                if (token['misc']['BeginSeg'] == 'Yes'):
                    sent_labels.append('Y')
                else:
                    sent_labels.append('N')
            else:
                sent_labels.append('N')
        labels.append(sent_labels)
    return labels

test_labels = get_labels(test_sent_list)
train_labels = get_labels(train_sent_list)

In [7]:
print(test_labels[0])

['Y', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']


In [8]:
def preprocess(lst): #убираем значения misc и значения feats переносим в словарь токена
    for elem in lst:
        for token in elem:
            del token['misc']
            d = {}
            for key in token:
                if (token[key] == None):
                    token[key] = '_ZERO_'
                elif (key == 'feats'):
                    for _ in token[key]:
                        d[_] = token[key][_]
            del token['feats']
            for _ in d:
                token[_] = d[_]
preprocess(test_sent_list)
preprocess(train_sent_list)

In [9]:
test_sent_list[2][1]

{'id': 2,
 'form': 'поездки',
 'lemma': 'поездка',
 'upos': 'NOUN',
 'xpos': '_ZERO_',
 'head': 22,
 'deprel': 'obl',
 'deps': '_ZERO_',
 'Animacу': 'Inan',
 'Case': 'Gen',
 'Gender': 'Fem',
 'Number': 'Sing'}

In [11]:
#! pip install --upgrade 'scikit-learn==0.23.2'

In [10]:
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=2.0,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(train_sent_list, train_labels)

/home/peter/p/tf-demo/lib/python3.9/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=2.0, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [11]:
from sklearn_crfsuite import metrics
labels_pred = crf.predict(test_sent_list)
metrics.flat_f1_score(test_labels, labels_pred,
                      average='weighted', labels=['Y'])

0.7390434624477178

## Домашнее задание (6 баллов)

Обучите модель для дискурсивной сегментации - CRF или бинарный классификатор (на выбор). Для оценки качества используйте F1 определения метки ``BeginSeg=Yes``.

Решения в форме ноутбуков, где последняя ячейка содержит ответ с текстовым комментарием, присылайте на mipttextanalysis20@gmail.com (ссылкой на colab.research или файлом). 

Будут приняты решения, присланные до 12:00 27 октября (без штрафа).

### Комментарий

Я перенес признаки feats в словарь токена и удалил словарь feats;
Также наличие метки BeginSeg=Yes я определил меткой 'Y', а меткой 'N' ее отстутствие. Отсутствие какого-либо признака (None) я заменил на флаг '_ ZERO _'